In [3]:
import time
from copy import deepcopy
import argparse
import sys
import traceback

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from utils import *


parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.seed = 123
args.nbits = 2048
args.n_splits = 5
args.test_size = 0.2
args.num_mol = 50000
args.max_len = 120
args.shuffle = True
args.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
args.optim = 'RMSProp'

np.random.seed(args.seed)
torch.manual_seed(args.seed)

# 1. Prepare Dataset

In [4]:
list_partition = make_partition(args)

# 2. Model Architecture

Create char_to_ix Ref: https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/word_embeddings_tutorial.py  
Pre-defined Embedding Layer Ref: https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76  
ResNet Variation Ref: https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035

In [5]:
def create_vocab(filename='../Data/logP/vocab.npy'):
    vocab = np.load(filename)
    vocab_size = len(vocab)
    char_to_ix = {char: i for i, char in enumerate(vocab)}
    return vocab, char_to_ix

In [6]:
class ResBlock(nn.Module):
    def __init__(self, in_filter, out_filter, stride, use_bn, dp_rate, block_type):
        super(ResBlock, self).__init__()   
        self.use_bn = use_bn
        self.block_type = block_type
        self.conv1 = nn.Conv2d(in_filter, out_filter, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_filter, out_filter, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(out_filter)
        self.bn2 = nn.BatchNorm2d(out_filter)
        self.dropout = nn.Dropout2d(dp_rate)
        self.shortcut = nn.Sequential()
        if in_filter != out_filter:
            self.shortcut.add_module(
                'conv', nn.Conv2d(in_filter, out_filter,
                                  kernel_size=1, stride=stride, 
                                  padding=0, bias=False)
            )
        
    def forward(self, _x):
        if self.block_type == 'a': #original residual block
            x = self.relu(self.bn1(self.conv1(_x))) if self.use_bn else self.relu(self.conv1(_x))
            x = self.bn2(self.conv2(x)) if self.use_bn else self.conv2(x)
            x = x + self.shortcut(_x)
            return self.dropout(self.relu(x))
        
        elif self.block_type == 'b': # BN after addition
            x = self.relu(self.bn1(self.conv1(_x))) if self.use_bn else self.relu(self.conv1(_x))
            x = self.conv2(x) + self.shortcut(_x)
            return self.dropout(self.relu(self.bn2(x)) if self.use_bn else self.relu(x))
        
        elif self.block_type == 'c': # ReLU before addition
            x = self.relu(self.bn1(self.conv1(_x))) if self.use_bn else self.relu(self.conv1(_x))
            x = self.relu(self.bn2(self.conv2(x))) if self.use_bn else self.relu(self.conv2(x))
            return self.dropout(x + self.shortcut(_x))
        
        elif self.block_type == 'd': # ReLU-only pre-activation
            x = self.bn1(self.conv1(self.relu(_x))) if self.use_bn else self.conv1(self.relu(_x))
            x = self.bn2(self.conv2(self.relu(x))) if self.use_bn else self.conv2(self.relu(x))
            return self.dropout(x + self.shortcut(_x))
        
        elif self.block_type == 'e': # full pre-activation
            x = self.conv1(self.relu(self.bn1(_x))) if self.use_bn else self.conv1(self.relu(_x))
            x = self.conv2(self.relu(self.bn2(x))) if self.use_bn else self.conv2(self.relu(x))
            return self.dropout(x + self.shortcut(_x))
             
            

class Net(nn.Module):
    def __init__(self, args):
        super(Net, self).__init__()   
        list_n_filter = [32*2**i for i in range(args.n_layer)]
        
        self.embedding = self.create_emb_layer(args.vocab_size, args.emb_train)
        list_res_blocks = [
            ResBlock(1, 32, args.stride, args.use_bn, args.dp_rate, args.block_type)
        ]
        self.res_blocks = nn.Sequential(*list_res_blocks)
        self.fc = nn.Linear(122880, 1)

    def forward(self, x):
        embeds = self.embedding(x)
        embeds = embeds.view(embeds.shape[0], 1, embeds.shape[1], embeds.shape[2])
        x = self.res_blocks(embeds)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return torch.squeeze(x)

    def create_emb_layer(self, vocab_size, emb_train):
        emb_layer = nn.Embedding(vocab_size, vocab_size)
        weight_matrix = torch.zeros((vocab_size, vocab_size))
        for i in range(vocab_size):
            weight_matrix[i][i] = 1
        emb_layer.load_state_dict({'weight': weight_matrix})

        if not emb_train:
            emb_layer.weight.requires_grad = False
        return emb_layer

In [11]:
def train(model, partition, optimizer, criterion, char_to_ix, args):
    data_iter = DataLoader(
        partition['train'],
        batch_size=args.batch_size,
        shuffle=args.shuffle
    )
    
    epoch_train_loss = 0
    cnt_iter = 0
    for batch_idx, batch in enumerate(data_iter):
        X, y = batch[0], batch[1]
        X = torch.Tensor([[char_to_ix[c] for c in smile] for smile in X]).long()
        X, y = X.to(args.device), y.to(args.device).float()
    
        model.train()
        optimizer.zero_grad()

        pred_y = model(X)
        pred_y.require_grad = False
        train_loss = criterion(pred_y, y)
        epoch_train_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()

        cnt_iter += 1

    return model, epoch_train_loss/cnt_iter

def validate(model, partition, criterion, char_to_ix, args):
    data_iter = DataLoader(
        partition['val'],
        batch_size=args.test_batch_size,
        shuffle=args.shuffle
    )
    
    epoch_val_loss = 0
    cnt_iter = 0
    for batch_idx, batch in enumerate(data_iter):
        X, y = batch[0], batch[1]
        X = torch.Tensor([[char_to_ix[c] for c in smile] for smile in X]).long()
        X, y = X.to(args.device), y.to(args.device).float()
    
        model.eval()
        pred_y = model(X)
        pred_y.require_grad = False
        val_loss = criterion(pred_y, y)
        epoch_val_loss += val_loss.item()
        cnt_iter += 1

    return epoch_val_loss/cnt_iter

def test(model, partition, char_to_ix, args):
    data_iter = DataLoader(
        partition['test'],
        batch_size=args.test_batch_size,
        shuffle=args.shuffle
    )
    
    cnt_iter = 0
    list_y, list_pred_y = list(), list()
    for batch_idx, batch in enumerate(data_iter):
        X, y = batch[0], batch[1]
        X = torch.Tensor([[char_to_ix[c] for c in smile] for smile in X]).long()
        X, y = X.to(args.device), y.to(args.device).float()
    
        model.eval()
        pred_y = model(X)
        list_y += y.cpu().detach().numpy().tolist()
        list_pred_y += pred_y.cpu().detach().numpy().tolist()
        cnt_iter += 1

    mae = mean_absolute_error(list_y, list_pred_y)
    std = np.std(np.array(list_y)-np.array(list_pred_y))
    return mae, std

def experiment(partition, args):
    vocab, char_to_ix = create_vocab()
    args.vocab_size = len(vocab)
    args.input_shape = (args.max_len, args.vocab_size)
    model = Net(args)
    model.to(args.device)
    criterion = nn.MSELoss()
    
    # Initialize Optimizer
    trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
    if args.optim == 'ADAM':
        optimizer = optim.Adam(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    elif args.optim == 'RMSProp':
        optimizer = optim.RMSprop(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    elif args.optim == 'SGD':
        optimizer = optim.SGD(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    else:
        assert False, "Undefined Optimizer Type"
        
    for epoch in range(args.epoch):
        model, train_loss = train(model, partition, optimizer, criterion, char_to_ix, args)
        val_loss = validate(model, partition, criterion, char_to_ix, args)
        mae, std = test(model, partition, char_to_ix, args)
        
        print(train_loss, val_loss, mae, std)
        
    torch.cuda.empty_cache()

In [15]:
args.exp_name = 'exp_hidden_dim'
args.n_layer = 2
args.lr = 0.001
args.l2_coef = 0
args.optim = 'RMSProp'
args.epoch = 1
args.batch_size=256
args.test_batch_size=1024
args.emb_train = False
args.stride = 1
args.use_bn = True
args.dp_rate = 0
args.block_type = 'a'
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment(list_partition[0], args)

165.41904091644287 3.1543420553207397 1.5763102052140865 0.8701013226664565
